In [24]:
import pandas as pd

df = pd.read_csv('/Users/hamidrezakomaki/Desktop/CS_Stuffs/CS5800/CS800_env/UNSPSC_English.csv', low_memory=False)
pd.set_option('display.max_columns', 17)

In [ ]:
pd.set_option('display.max_rows', 100)

In [39]:
df = df.drop(columns='Unnamed: 16')

In [75]:
df = df.drop(index=8)

In [56]:
df = df.drop(columns='UNSPSC v23_0701')

In [76]:
df

,Segment Title,Segment Definition,Family Title,Family Definition,Class Title,Class Definition,Commodity Title,Definition
9,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",NaN,NaN,NaN,NaN,NaN,NaN
10,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",Live animals,NaN,NaN,NaN,NaN,NaN
11,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",Live animals,NaN,Livestock,NaN,NaN,NaN
12,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",Live animals,NaN,Livestock,NaN,Cats,NaN
13,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",Live animals,NaN,Livestock,NaN,Dogs,NaN
...,...,...,...,...,...,...,...,...
156402,Land and Buildings and Structures and Thorough...,Land and waterways and related real property p...,Prefabricated buildings and structures,Buildings and structures used for various farm...,Prefabricated medical buildings and structures,Buildings and structures used for medical purp...,NaN,NaN
156403,Land and Buildings and Structures and Thorough...,Land and waterways and related real property p...,Prefabricated buildings and structures,Buildings and structures used for various farm...,Prefabricated medical buildings and structures,Buildings and structures used for medical purp...,Medical unit,A building or structure where medical equipmen...
156404,Land and Buildings and Structures and Thorough...,Land and waterways and related real property p...,Prefabricated buildings and structures,Buildings and structures used for various farm...,Prefabricated medical buildings and structures,Buildings and structures used for medical purp...,Laboratory unit,A building or structure where laboratory equip...
156405,Land and Buildings and Structures and Thorough...,Land and waterways and related real property p...,Prefabricated buildings and structures,Buildings and structures used for various farm...,Prefabricated medical buildings and structures,Buildings and structures used for medical purp...,Dental unit,A building or structure where dental equipment...


In [74]:
df = df.rename(columns={'Unnamed: 14':'Definition'})

In [79]:
df = df.reset_index()

In [81]:
df = df.drop(columns='index')

In [83]:
df.sample(n=20)

,Segment Title,Segment Definition,Family Title,Family Definition,Class Title,Class Definition,Commodity Title,Definition
94577,Healthcare Services,This segment includes services associated with...,Surgical interventions or procedures of lower ...,This family of surgical interventions or proce...,"Principal arteries abdomen, open approach-part F",This class includes one or more of 17 standard...,Dilation of splenic artery with drug-eluting i...,This classification denotes the group of activ...
73999,Musical Instruments and Games and Toys and Art...,This Segment includes teaching aids used in bo...,Developmental and professional teaching aids a...,NaN,Educational incentives,NaN,Bible based incentive charts,NaN
124231,Healthcare Services,This segment includes services associated with...,Surgical interventions or procedures of upper ...,This family of surgical interventions or proce...,"Flat bones, percutaneous endoscopic approach-p...",This class includes one or more of 17 standard...,"Excision of left scapula, percutaneous endosc...",This classification denotes the group of activ...
139075,Healthcare Services,This segment includes services associated with...,Surgical interventions or procedures of female...,This family of surgical interventions or proce...,"Uterus cervix fallopian tubes, percutaneous en...",This class includes one or more of 23 standard...,"Excision of uterus, percutaneous endoscopic a...",This classification denotes the group of activ...
155000,Healthcare Services,This segment includes services associated with...,Physical rehabilitation interventions or proce...,This family of physical rehabilitation interve...,Daily living activities-mobility and occupatio...,This class includes one or more of multiple st...,"Environmental, home and work barriers assessme...",This classification denotes the group of activ...
131537,Healthcare Services,This segment includes services associated with...,Surgical interventions or procedures of upper ...,This family of surgical interventions or proce...,"Joints hand, percutaneous endoscopic approach-...",This class includes one or more of 16 standard...,Fusion of right metacarpocarpal joint with ext...,This classification denotes the group of activ...
97643,Healthcare Services,This segment includes services associated with...,Surgical interventions or procedures of lower ...,This family of surgical interventions or proce...,"Principal arteries lower extremities, percutan...",This class includes one or more of 17 standard...,Occlusion of right femoral artery with extralu...,This classification denotes the group of activ...
24444,Medical Equipment and Accessories and Supplies,"This segment includes the tools and machines, ...",Patient exam and monitoring products,"In this classification, this family includes m...",Ophthalmic diagnostic exam products,"In this classification, this class includes, p...",Ophthalmic slit lamps,"In this classification, this commodity indicat..."
68924,Drugs and Pharmaceutical Products,This segment includes natural or synthetic mat...,Antifungal drugs,This classification denotes substances that de...,Antifungal amines,Antifungal drugs made from a group of compound...,Butenafine,This classification denotes an antifungal agen...
11966,Commercial and Military and Private Vehicles a...,This segment includes vehicles used in the con...,Motor vehicles,NaN,Product and material transport vehicles,NaN,Load motovan,Special motor vehicle usually of three wheels ...


In [85]:
df.to_csv('my_DataFrame.csv')

,Segment Title,Segment Definition,Family Title,Family Definition,Class Title,Class Definition,Commodity Title,Definition
0,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",NaN,NaN,NaN,NaN,NaN,NaN
1,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",Live animals,NaN,NaN,NaN,NaN,NaN
2,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",Live animals,NaN,Livestock,NaN,NaN,NaN
3,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",Live animals,NaN,Livestock,NaN,Cats,NaN
4,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",Live animals,NaN,Livestock,NaN,Dogs,NaN
...,...,...,...,...,...,...,...,...
156393,Land and Buildings and Structures and Thorough...,Land and waterways and related real property p...,Prefabricated buildings and structures,Buildings and structures used for various farm...,Prefabricated medical buildings and structures,Buildings and structures used for medical purp...,NaN,NaN
156394,Land and Buildings and Structures and Thorough...,Land and waterways and related real property p...,Prefabricated buildings and structures,Buildings and structures used for various farm...,Prefabricated medical buildings and structures,Buildings and structures used for medical purp...,Medical unit,A building or structure where medical equipmen...
156395,Land and Buildings and Structures and Thorough...,Land and waterways and related real property p...,Prefabricated buildings and structures,Buildings and structures used for various farm...,Prefabricated medical buildings and structures,Buildings and structures used for medical purp...,Laboratory unit,A building or structure where laboratory equip...
156396,Land and Buildings and Structures and Thorough...,Land and waterways and related real property p...,Prefabricated buildings and structures,Buildings and structures used for various farm...,Prefabricated medical buildings and structures,Buildings and structures used for medical purp...,Dental unit,A building or structure where dental equipment...
